In [1]:
# based on this guide: https://www.tensorflow.org/get_started/eager

from __future__ import absolute_import, division, print_function

import os
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.contrib.eager as tfe

#tf.enable_eager_execution() # must be run on a start up program

print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

import datetime
import pandas as pd
import numpy as np
import random

Instructions for updating:
Use the retry module or similar alternatives.
TensorFlow version: 1.7.0
Eager execution: False


In [2]:


# load in the data
# replace by owrn own parsing

metadata_file = '/Users/jaspermeijering/Google Drive/a Study/EPA Study Abroad - Carnegie Mellon University/Courses/CMU - 95845 - Applied Analytics The Machine Learning Pipeline/Machine Learning Pipeline Final Project/Data/FalseNews_Code_Data/data/metadata_anon.txt'

# Read meta data 
fin = open(metadata_file,'r')
lines = fin.readlines()
fin.close()
cascade_id2metadata={}
for line in lines:
    line = line.replace('\n','')
    item = eval(line)
    cascade_id2metadata[item[0]] = item[1]

    # Get static measures
veracity = []
virality = []
depth = []
breadth = []
size = []
verified = []
nfollowers = []
nfollowees = []
engagement = []
category = []
day = []
hour = []

for cascade,metadata in cascade_id2metadata.items():
    veracity.append(metadata['veracity'])
    virality.append(metadata['virality'])
    depth.append(metadata['depth'])
    breadth.append(metadata['max_breadth'])
    size.append(metadata['size'])
    verified.append(metadata['verified_list'][0])
    nfollowers.append(metadata['num_followers_list'][0])
    nfollowees.append(metadata['num_followees_list'][0])
    engagement.append(metadata['engagement_list'][0])
    category.append(metadata['rumor_category'])
    day.append(metadata['start_date'].day)
    hour.append(metadata['start_date'].hour)

# Convert to data frame
df = pd.DataFrame({'veracity': veracity, 'virality': virality, 'depth': depth, 'breadth': breadth, 'size': size, 'verified': verified, 'nfollowers': nfollowers, 
                   'nfollowees': nfollowees, 'engangement': engagement, 'category': category, 'day': day, 'hour': hour})

# Inspect
df.head(5)


#def parse_data(line):
 #   example_defaults = [[0.], [0.], [0.], [0.], [0]]  # sets field types
  #features = tf.reshape(parsed_line[:-1], shape=(4,))
   # # Last field is the label
  #  label = tf.reshape(parsed_line[-1], shape=())
  #  return features, label



,breadth,category,day,depth,engangement,hour,nfollowees,nfollowers,size,veracity,verified,virality
0,1,Politics,22,0,7.092405,21,236.0,164.0,1,FALSE,False,NaN
1,1,Politics,23,0,8.394500,14,1999.0,870.0,1,FALSE,False,NaN
2,1,Politics,9,0,6.729679,16,1982.0,452.0,1,FALSE,False,NaN
3,1,Politics,15,0,0.186418,20,55.0,5.0,1,FALSE,False,NaN
4,1,Politics,14,0,1.425915,14,129.0,95.0,1,FALSE,False,NaN


In [3]:
# df["veracity"] = df["veracity"].astype('int') Doens't work

In [4]:
df["veracity"] = df["veracity"].astype('category')
df["category"] = df["category"].astype('category')


In [5]:
#df_x = df[["breadth","category","day","depth","engangement","hour","nfollowees","nfollowers","size","veracity","verified"]]
df_x = df[["breadth","day","category","depth","engangement","hour","nfollowees","nfollowers","size","verified"]]
df_y = df[['veracity']] 
df_x.head(10)

,breadth,day,category,depth,engangement,hour,nfollowees,nfollowers,size,verified
0,1,22,Politics,0,7.092405,21,236.0,164.0,1,False
1,1,23,Politics,0,8.394500,14,1999.0,870.0,1,False
2,1,9,Politics,0,6.729679,16,1982.0,452.0,1,False
3,1,15,Politics,0,0.186418,20,55.0,5.0,1,False
4,1,14,Politics,0,1.425915,14,129.0,95.0,1,False
5,1,18,Viral Photos/Stories/Urban Legends,0,2.299674,10,353.0,175.0,1,False
6,1,18,War/Terrorism/Shootings,0,15.413659,16,993.0,18044.0,1,False
7,1,13,Viral Photos/Stories/Urban Legends,0,0.057485,9,86.0,33.0,1,False
8,1,17,War/Terrorism/Shootings,0,11.152556,16,1230.0,931.0,1,False
9,1,14,Viral Photos/Stories/Urban Legends,0,27.463058,11,994.0,1442.0,1,False


## Running the model with non numerical columns does not work therefore (for now) only take the numerical variables (and use day as the y).

In [6]:
df_x_test = df[["breadth","depth","engangement","hour","nfollowees","nfollowers","size"]]
df_y_test = df[['day']]  #does not work in model

In [7]:
df_y.head(10) # does not work in model

,veracity
0,FALSE
1,FALSE
2,FALSE
3,FALSE
4,FALSE
5,TRUE
6,TRUE
7,TRUE
8,TRUE
9,TRUE


In [8]:
#df_x["category"] = df_x["category"].astype('category')
#df_y.loc[:,1] = df_y["veracity"].astype('category')

In [9]:
#df_x["category"] = df_x["category"].astype('category')
df_x  = df_x.as_matrix()
df_y = df_y.as_matrix()

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Activation

Using TensorFlow backend.


In [11]:
# For a single-input model with 2 classes (binary classification):

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Generate dummy data
import numpy as np
data = df_x_test
labels = df_y_test

# Train the model, iterating on the data in batches of 32 samples
model.fit(data, labels, epochs=10, batch_size=32)

Epoch 1/10
126301/126301 [==============================] - 4s 34us/step - loss: nan - acc: 3.9588e-05
Epoch 2/10
126301/126301 [==============================] - 4s 34us/step - loss: nan - acc: 0.0000e+00
Epoch 3/10
126301/126301 [==============================] - 4s 34us/step - loss: nan - acc: 0.0000e+00
Epoch 4/10
126301/126301 [==============================] - 4s 34us/step - loss: nan - acc: 0.0000e+00
Epoch 5/10
126301/126301 [==============================] - 4s 35us/step - loss: nan - acc: 0.0000e+00
Epoch 6/10
126301/126301 [==============================] - 4s 34us/step - loss: nan - acc: 0.0000e+00
Epoch 7/10
126301/126301 [==============================] - 4s 34us/step - loss: nan - acc: 0.0000e+00
Epoch 8/10
126301/126301 [==============================] - 4s 35us/step - loss: nan - acc: 0.0000e+00
Epoch 9/10
126301/126301 [==============================] - 4s 35us/step - loss: nan - acc: 0.0000e+00
Epoch 10/10
126301/126301 [==============================] - 5s 36us/step

In [14]:
loss_and_metrics = model.evaluate(df_x_test, df_y_test, batch_size=128) # first split train and test set (code is elsewhere)
#loss_and_metrics = model.evaluate(x_test, y_test, batch_size=128)
loss_and_metrics

126301/126301 [==============================] - 1s 5us/step


[nan, 0.0]

### Stacked LSTM for sequence classification
In this model, we stack 3 LSTM layers on top of each other, making the model capable of learning higher-level temporal representations.

The first two LSTMs return their full output sequences, but the last one only returns the last step in its output sequence, thus dropping the temporal dimension (i.e. converting the input sequence into a single vector).



In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

data_dim = 16
timesteps = 8
num_classes = 10

# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(32, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32))  # return a single vector of dimension 32
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Generate dummy training data
x_train = np.random.random((1000, timesteps, data_dim))
y_train = np.random.random((1000, num_classes))

# Generate dummy validation data
x_val = np.random.random((100, timesteps, data_dim))
y_val = np.random.random((100, num_classes))

model.fit(x_train, y_train,
          batch_size=64, epochs=5,
          validation_data=(x_val, y_val))